In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from utils.load_brad_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.linear_regression import SimpleLinearReg, SimpleFitCurve
from models.concurrency.xgboost import XGBoostPredictor
np.set_printoptions(suppress=True)

In [ ]:
#directory = "/Users/ziniuw/Desktop/research/Data/BRAD_trace/brad_dl_traces/brad_del_trace.task.1706423438/trace/"
#directory = "/Users/ziniuw/Desktop/research/Data/BRAD_trace/brad_dl_traces/brad_del_trace.task.1706488009/trace/"
directory = "/Users/ziniuw/Desktop/research/Data/BRAD_trace/concurrent_3_brad/"
directory = "/Users/ziniuw/Desktop/research/Data/BRAD_trace/io_heavy_redshift/"
raw_trace, trace = load_trace(directory, concat=True)
concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=60)
isolated_trace_df = pd.read_csv("/Users/ziniuw/Desktop/research/Data/BRAD_trace/io_heavy_redshift/repeating_olap_batch_warmup.csv")
isolated_trace_df["runtime"] = isolated_trace_df["run_time_s"]
isolated_rt_cache = dict()
for i, rows in isolated_trace_df.groupby("query_idx"):
    isolated_rt_cache[i] = np.median(rows["runtime"])

In [2]:
folder_name = "mixed_aurora"
directory = f"/Users/ziniuw/Desktop/research/Data/BRAD_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 4, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=60)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)
isolated_trace_df = pd.read_csv(f"/Users/ziniuw/Desktop/research/Data/BRAD_trace/{folder_name}/repeating_olap_batch_warmup.csv")
isolated_trace_df["runtime"] = isolated_trace_df["run_time_s"]
isolated_rt_cache = dict()
for i, rows in isolated_trace_df.groupby("query_idx"):
    isolated_rt_cache[i] = np.median(rows["runtime"])

In [ ]:
for i, rows in isolated_trace_df.groupby("query_idx"):
    runtime = rows["runtime"].values
    print(i, len(rows), np.mean(runtime), np.min(runtime), np.max(runtime), np.std(runtime))

In [3]:
for i, rows in concurrency_df.groupby("query_idx"):
    runtime = rows["runtime"].values
    print(i, len(rows), isolated_rt_cache[i], np.mean(runtime), np.min(runtime), np.max(runtime), np.std(runtime))
    #if i == 10:
     #   break
    #plt.hist(runtime, bins=60)
    #plt.xlabel("Query runtime (s)", fontsize=20)
    #plt.ylabel("Number of queries", fontsize=20)
    #plt.savefig(f"query_{i}.pdf")
    #plt.show()

0 161 44.55030298233032 122.73418354247667 3.882469892501831 472.2169952392578 110.62397192713661
1 164 0.2543864250183105 0.6545542042429854 0.0791635513305664 7.855563879013061 1.1357284066670006
2 164 23.157404899597168 152.27174959095512 6.902190923690796 868.5123317241669 142.2654456745285
3 165 0.283794641494751 0.29162575114857064 0.0799896717071533 1.5334429740905762 0.25116029490832664
4 165 4.330385208129883 22.801687944296635 0.291949987411499 173.76991820335388 30.611828568937465
5 165 2.5214909315109253 15.38704783266241 2.5013554096221924 183.3675537109375 26.603118081575424
6 163 2.5594241619110107 3.4007910585110905 2.5194125175476074 10.34561848640442 1.1073853839347692
7 164 2.2192370891571045 9.77907490148777 1.766042709350586 114.38352727890016 17.93298391949548
8 165 0.3999779224395752 1.1193671370997573 0.1878495216369629 18.592463493347168 2.3370285947114624
9 165 0.7344051599502563 0.12170190088676681 0.0426421165466308 8.005128622055054 0.6233252422485591
10 16

In [4]:
np.random.seed(4)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
eval_trace_df = concurrency_df.iloc[test_idx]
eval_trace_df = eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0]

slr = SimpleLinearReg()
slr.train(concurrency_df.iloc[train_idx], isolated_trace_df=isolated_trace_df)
#predictions, labels = slr.predict(eval_trace_df)
#print("===========Performance for simple linear regression model (per query)=============")
#result_overall, result_per_query = slr.evaluate_performance(eval_trace_df, use_train=True)
predictions_slr, labels = slr.predict(eval_trace_df, use_global=True)
print("===========Performance for simple linear regression model (all query)=============")
result_overall_slr, result_per_query_slr = slr.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple linear regression model (all query)=============
50% absolute error is 17.886651178540323, q-error is 3.95583854493825
90% absolute error is 125.83290213576903, q-error is 10.05229418975591
95% absolute error is 193.8505202728002, q-error is 15.41365347364018


In [5]:
slr = SimpleLinearReg()
slr.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df)
#predictions, labels = slr.predict(eval_trace_df, use_train=False)
#print("===========Performance for simple linear regression model (per query & train on Full)=============")
#result_overall, result_per_query = slr.evaluate_performance(eval_trace_df, use_train=False)
predictions_slr, labels = slr.predict(eval_trace_df, use_global=True)
print("===========Performance for simple linear regression model (all query & train on Full)=============")
result_overall_slr, result_per_query_slr = slr.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple linear regression model (all query & train on Full)=============
50% absolute error is 15.416693939464594, q-error is 3.675736799528974
90% absolute error is 141.72244322549676, q-error is 9.106786436002837
95% absolute error is 267.55782928091156, q-error is 13.481654843422914


In [45]:
sfc = SimpleFitCurve()
sfc.train(concurrency_df.iloc[train_idx], isolated_trace_df=isolated_trace_df)
#predictions, labels = sfc.predict(eval_trace_df)
#print("===========Performance for simple curve fitting model (per query)=============")
#result_overall, result_per_query = sfc.evaluate_performance(eval_trace_df, use_train=True)
predictions_sfc, labels = sfc.predict(eval_trace_df, use_global=True)
print("===========Performance for simple curve fitting model (all query)=============")
result_overall_sfc, result_per_query_sfc = sfc.evaluate_performance(eval_trace_df, use_global=True)

===========Performance for simple curve fitting model (all query)=============
50% absolute error is 9.566637866526179, q-error is 2.8127277928809793
90% absolute error is 98.80636927733629, q-error is 7.548735018799135
95% absolute error is 158.2199729233719, q-error is 11.821560038303017


In [46]:
sfc = SimpleFitCurve()
sfc.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df)
#predictions, labels = sfc.predict(eval_trace_df, use_train=False)
#print("===========Performance for simple curve fitting model (per query)=============")
predictions_sfc, labels = sfc.predict(eval_trace_df, use_global=True)
print("===========Performance for simple curve fitting model (all query & train on full)=============")
result_overall_sfc, result_per_query_sfc = sfc.evaluate_performance(eval_trace_df, use_global=False)

===========Performance for simple curve fitting model (all query & train on full)=============
50% absolute error is 10.612637319194409, q-error is 2.2545820040861098
90% absolute error is 67.47848973705275, q-error is 6.923099111719248
95% absolute error is 108.85602286516045, q-error is 10.201603682363771


In [29]:
import numpy as np
from models.concurrency.base_model import ConcurPredictor
from xgboost import XGBRegressor


class XGBoostPredictor(ConcurPredictor):
    """
    Consider k unique concurrent queries (or group into k classes)
    For each query represent it as k-dimensional one hot vector
         Then, identify its concurrent queries, represent them as a k-dimensional vector
         [rt_q1, rt_q2, ..., rt_qk], where rt_qi is the runtime of query i running in isolation
         Concatenate two features together to build simple XGboost model to predict the runtime
    """
    def __init__(self, k=100):
        self.clustering = None
        self.isolated_rt_cache = dict()
        self.xgboost = None
        self.use_train = True
        self.use_pre_exec_info = False
        self.k = k

    def train(self, trace_df, use_train=True, isolated_trace_df=None, use_pre_exec_info=False):
        self.use_train = use_train
        self.use_pre_exec_info = use_pre_exec_info
        self.get_isolated_runtime_cache(trace_df, isolated_trace_df)
        concurrent_df = trace_df[trace_df['num_concurrent_queries'] > 0]

        global_y = []
        global_x = []
        for i, rows in concurrent_df.groupby("query_idx"):
            if i not in self.isolated_rt_cache or len(rows) < 2:
                continue
            if use_train:
                concur_info = rows["concur_info_train"].values
            else:
                concur_info = rows["concur_info"].values
            global_y.append(rows["runtime"].values)
            query_feature = np.zeros((len(rows), self.k))
            query_feature[:, i] = self.isolated_rt_cache[i]
            #query_feature[:, i] = 1
            concur_query_feature = np.zeros((len(rows), self.k))
            for j in range(len(rows)):
                for c in concur_info[j]:
                    if c[0] in self.isolated_rt_cache:
                        concur_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                        #concur_query_feature[j, c[0]] += 1
                    else:
                        concur_query_feature[j, c[0]] += 1
            x = np.concatenate((query_feature, concur_query_feature), axis=1)
            if use_pre_exec_info:
                pre_exec_query_feature = np.zeros((len(rows), self.k))
                pre_exec_info = rows["pre_exec_info"].values
                for j in range(len(rows)):
                    for c in pre_exec_info[j]:
                        if c[0] in self.isolated_rt_cache:
                            pre_exec_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                            #concur_query_feature[j, c[0]] += 1
                        else:
                            pre_exec_query_feature[j, c[0]] += 1
                x = np.concatenate((x, pre_exec_query_feature), axis=1)
            global_x.append(x)
        global_y = np.concatenate(global_y)
        global_x = np.concatenate(global_x)
        model = XGBRegressor(n_estimators=1000, max_depth=8, eta=0.2, subsample=1.0,
                             eval_metric="mae", early_stopping_rounds=100)
        train_idx = np.random.choice(len(global_y), size=int(0.8 * len(global_y)), replace=False)
        val_idx = [i for i in range(len(global_y)) if i not in train_idx]
        model.fit(global_x[train_idx], global_y[train_idx],
                  eval_set=[(global_x[val_idx], global_y[val_idx])],
                  verbose=False
                 )
        self.xgboost = model


    def predict(self, eval_trace_df, use_global=True):
        predictions = dict()
        labels = dict()
        for i, rows in eval_trace_df.groupby("query_idx"):
            if i not in self.isolated_rt_cache or len(rows) < 2:
                continue
            label = rows["runtime"].values
            labels[i] = label
            if self.use_train:
                concur_info = rows["concur_info_train"].values
            else:
                concur_info = rows["concur_info"].values
            query_feature = np.zeros((len(rows), self.k))
            query_feature[:, i] = self.isolated_rt_cache[i]

            concur_query_feature = np.zeros((len(rows), self.k))
            for j in range(len(rows)):
                for c in concur_info[j]:
                    if c[0] in self.isolated_rt_cache:
                        concur_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                    else:
                        concur_query_feature[j, c[0]] += 2
            x = np.concatenate((query_feature, concur_query_feature), axis=1)
            if self.use_pre_exec_info:
                pre_exec_query_feature = np.zeros((len(rows), self.k))
                pre_exec_info = rows["pre_exec_info"].values
                for j in range(len(rows)):
                    for c in pre_exec_info[j]:
                        if c[0] in self.isolated_rt_cache:
                            pre_exec_query_feature[j, c[0]] += self.isolated_rt_cache[c[0]]
                            #concur_query_feature[j, c[0]] += 1
                        else:
                            pre_exec_query_feature[j, c[0]] += 1
                x = np.concatenate((x, pre_exec_query_feature), axis=1)
            #if i == 0:
             #   for k in range(len(label)):
              #      print(x[k], label[k])
                
            pred = self.xgboost.predict(x)
            pred = np.maximum(pred, 0.001)
            predictions[i] = pred
        return predictions, labels



In [21]:
xgb = XGBoostPredictor(k=240)
xgb.train(concurrency_df.iloc[train_idx], isolated_trace_df=isolated_trace_df, use_pre_exec_info=True)
predictions_xgb, labels = xgb.predict(eval_trace_df)
print("===========Performance for XGBoost model=============")
result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(eval_trace_df)

===========Performance for XGBoost model=============
50% absolute error is 13.823158264160156, q-error is 3.0152116705476817
90% absolute error is 80.44210176467898, q-error is 42.48723870472399
95% absolute error is 131.48118872642516, q-error is 169.13333199814315


In [32]:
xgb = XGBoostPredictor(k=240)
xgb.train(concurrency_df.iloc[train_idx], use_train=False, isolated_trace_df=isolated_trace_df, use_pre_exec_info=True)
predictions_xgb, labels = xgb.predict(eval_trace_df)
#predictions_xgb, labels = xgb.predict(concurrency_df.iloc[train_idx], use_train=False)
print("===========Performance for XGBoost model (train on full)=============")
result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(eval_trace_df)
#result_overall_xgb, result_per_query_xgb = xgb.evaluate_performance(concurrency_df.iloc[train_idx], use_train=False)

===========Performance for XGBoost model (train on full)=============
50% absolute error is 9.91926097869873, q-error is 2.5930281679536393
90% absolute error is 62.981975412368776, q-error is 334.2335065399217
95% absolute error is 93.38124718666073, q-error is 1202.1658326401764


In [22]:
i = 0
print(isolated_trace_df["runtime"].iloc[i])
idx = np.argsort(predictions_xgb[i])
np.stack((predictions_xgb[i][idx], labels[i][idx]), axis=1)

29.34782457351685


array([[ 53.40248871,  29.3566246 ],
       [ 66.54302216,  12.10794377],
       [ 81.46742249, 187.10900164],
       [ 81.63794708,  59.03586602],
       [ 83.22177887, 213.94148636],
       [ 89.02323914,  46.65917611],
       [ 92.61101532, 211.4788456 ],
       [ 94.13135529, 130.59745121],
       [ 96.02250671,  31.13634539],
       [ 96.24590302, 119.18054438],
       [ 96.55078888,  12.576473  ],
       [ 97.92541504, 321.95021749],
       [ 98.68902588, 108.15937161],
       [100.44212341, 282.57655692],
       [104.90145111,  72.18599916],
       [106.04280853,  50.05959463],
       [109.27822113, 265.26644182],
       [110.86017609,  66.66565442],
       [111.18930817,  39.24779963],
       [115.18584442,   7.53865743],
       [117.65393066, 472.21699524],
       [124.88314819, 273.28928781],
       [125.88470459, 201.23098445],
       [129.34068298,  57.34541178],
       [130.0255127 , 174.01957989],
       [130.26077271, 306.62545347],
       [130.80278015,  45.72286344],
 

In [33]:
i = 0
print(isolated_trace_df["runtime"].iloc[i])
idx = np.argsort(predictions_xgb[i])
np.stack((predictions_xgb[i][idx], labels[i][idx]), axis=1)

29.34782457351685


array([[ 12.59923935,  46.65917611],
       [ 18.07102394,  31.13634539],
       [ 19.03647232,  29.3566246 ],
       [ 33.9233551 ,   7.53865743],
       [ 37.8103981 ,  12.10794377],
       [ 38.03136444,  50.05959463],
       [ 41.34118652,  39.24779963],
       [ 44.89187622,  45.72286344],
       [ 46.23538971,  17.26033831],
       [ 60.51521301, 306.62545347],
       [ 63.91905594, 108.15937161],
       [ 67.07704926,  12.576473  ],
       [ 69.91146851,  59.03586602],
       [ 79.26138306, 321.95021749],
       [ 81.45046997,  57.34541178],
       [ 89.70726013, 131.94944143],
       [ 98.76960754,  66.66565442],
       [107.03543854,  72.18599916],
       [113.77780151, 119.18054438],
       [135.43991089, 174.01957989],
       [142.36148071, 130.59745121],
       [160.1865387 , 211.4788456 ],
       [166.87211609, 213.94148636],
       [166.98297119, 226.82893133],
       [172.18913269, 212.03328562],
       [195.57923889, 187.10900164],
       [206.3561554 , 273.28928781],
 

In [47]:
i = 203
print(isolated_trace_df["runtime"].iloc[i])
idx = np.argsort(predictions_xgb[i])
np.stack((predictions_xgb[i][idx], predictions_sfc[i][idx], labels[i][idx]), axis=1)

19.744221448898315


array([[  5.30993032,  20.51465382,   1.69816327],
       [  5.54645491,  34.05061503,   2.26599145],
       [ 13.33336067,  34.05061503,   2.6599803 ],
       [ 15.88251209,  34.05061503,  24.36821675],
       [ 18.38376617,  34.05061503,  10.09616756],
       [ 23.69589424,  69.00004169,  45.94498825],
       [ 28.19123268,  69.00004169,  57.89848971],
       [ 28.38432312,  57.3502328 ,  81.56031132],
       [ 29.13092041,  22.40080615,   2.57876277],
       [ 29.52492714,  57.3502328 ,  19.13332558],
       [ 34.63723373,  34.05061503,   4.32507086],
       [ 36.9041481 ,  45.70042392,  30.16556525],
       [ 39.30314636,  69.00004169,  41.74225688],
       [ 39.75701523,  34.05061503,   2.32424498],
       [ 52.34174728,  69.00004169, 127.81153822],
       [ 55.51959229, 127.24908612,  26.89863086],
       [ 59.33606339, 127.24908612, 650.7327652 ],
       [ 65.49890137,  45.70042392,  33.26331902],
       [ 70.39873505,  69.00004169,  55.44515276],
       [ 74.71154022,  69.00004

In [51]:
for t, m in zip(eval_trace_df.query(f"query_idx == {i}")["runtime"].values, eval_trace_df.query(f"query_idx == {i}")["concur_info_train"].values):
    print(t, m)

189.86344814300537 [(220, 13627.95221, 13679.435213139495), (22, 13655.821968, 13690.308793942993), (18, 13659.829635, 13695.983608817825)]
39.24779963493347 [(219, 18741.014544, 19017.320929040285), (172, 18766.330489, 18823.193407138504), (159, 18785.076494, 18833.19611789946)]
12.576472997665403 [(2, 22235.128267999997, 22383.084063288083)]
174.01957988739014 [(103, 23039.037527, 23989.871220265915), (84, 23777.60402, 23806.880481362838), (225, 23782.454592, 23807.069357167235)]
273.28928780555725 [(69, 6619.045245, 6701.262070723648), (183, 6624.842624, 6721.185475400375)]
57.34541177749634 [(2, 15344.375888, 15384.13872588681), (159, 15383.91379, 15409.991699469605)]
66.66565442085266 [(49, 19109.565077, 19186.22303145442), (224, 19166.915509, 19201.158049121077)]
17.260338306427002 [(22, 36309.529222, 36351.065623748655), (103, 36319.539158, 36924.1611012354), (210, 36331.241556, 36335.6824091189)]
29.356624603271484 [(187, 39420.536466, 39892.225743410505), (48, 39584.453833, 39

In [52]:
eval_trace_df.head(5)

,index,query_idx,runtime,start_time,end_time,pre_exec_info,concur_info,num_concurrent_queries,concur_info_train,num_concurrent_queries_train
2,2,135,3.479030,0.887088,4.366118,[],"[(29, 0.0, 110.210542678833), (143, 0.0, 3.958...",2,"[(29, 0.0, 110.210542678833), (143, 0.0, 3.958...",2
15,15,202,3.515862,81.851482,85.367344,"[(28, 7.382493, 22.881574134796146), (86, 26.2...","[(29, 0.0, 110.210542678833), (184, 28.56629, ...",4,"[(29, 0.0, 110.210542678833), (184, 28.56629, ...",3
21,21,224,16.836552,100.908025,117.744577,"[(123, 43.362241999999995, 52.878114001647944)...","[(29, 0.0, 110.210542678833), (221, 85.104832,...",7,"[(29, 0.0, 110.210542678833), (221, 85.104832,...",2
27,27,67,3.878542,118.184558,122.063100,"[(75, 6.449771, 69.67815859422302), (229, 55.7...","[(221, 85.104832, 134.55309516833495), (2, 106...",3,"[(221, 85.104832, 134.55309516833495), (2, 106...",3
31,31,188,0.721914,140.729550,141.451464,"[(12, 81.43333100000001, 82.86316703477479), (...","[(2, 106.568961, 184.1101613993988), (103, 129...",3,"[(2, 106.568961, 184.1101613993988), (103, 129...",3


In [ ]:
import os
import re
dir = "/Users/ziniuw/Desktop/research/Data/join-order-benchmark/"
all_queries = []
i = 0
name_map = dict()
for file in os.listdir(dir):
    if str(file)[0].isnumeric():
        name_map[i] = file.split(".sql")[0]
        file = os.path.join(dir, file)
        with open(file, "r") as f:
            sql = f.read()
        sql = re.sub(r"\s+", ' ', sql)
        all_queries.append(sql)
        i += 1

In [ ]:
len(all_queries)

In [ ]:
with open(os.path.join(dir, "imdb_job.sql"), "w") as f:
    for q in all_queries:
        f.write(q + "\n")

In [ ]:
name_map

In [40]:
np.concatenate((np.ones(2), [1]))

array([1., 1., 1.])